In [21]:
import torch
import torch.nn as nn

## 장단기 메모리(Long Short-Term Memory, LSTM)

가장 단순한 형태의 RNN을 `Vanilla RNN` 또는 `Simple RNN` 이라 한다.

LSTM은 이런 RNN의 한계를 극복하기 위해 나온 변형 중 하나이다.

### 장기 의존성 문제(the problem of Long-Term Dependencies)

RNN은 출력 결과가 이전의 계산 결과에 의존한다. 그러나 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점이 있다

RNN의 `시점(time step)이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상`이 발생하기 때문인데,

이때문에 시점이 충분히 긴 상황에서는 앞 시점의 정보가 전체 정보에 대한 영향력은 거의 의미가 없을 수도 있다

만약 시점의 앞 쪽에 가장 중요한 정보가 위치해 있고 RNN이 충분한 기억력을 가지고 있지 못하면 예측이 부정확 할 수 있다.

이를 `장기 의존성 문제(the problem of Long-Term Dependencies)`라고 한다

### LSTM 구조

![alt text](vaniila_rnn_and_different_lstm_ver2.png)

LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 저장함

따라서 은닉 상태(hidden state)를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태(cell state)라는 값을 추가

RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보임

셀 상태는 $C$로 전 시점의 셀 상태 $C_{t-1}$가 다음 시점의 셀 상태 $C_t$를 구하기 위한 입력으로서 사용

은닉 상태값과 셀 상태값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용, 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재

시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는데 이 값들을 가지고 게이트를 조절

시그모이드 함수를 $\sigma$, 하이퍼볼릭탄젠트 함수를 $tanh$

$W_{xi}$ , $W_{xg}$ , $W_{xf}$ , $W_{xo}$ 는 $x_t$와 함께 각 게이트에서 사용되는 4개의 가중치

$W_{hi}$ , $W_{hg}$ , $W_{hf}$ , $W_{ho}$ 는 $h_{t-1}$와 함께 각 게이트에서 사용되는 4개의 가중치

$b_i$ , $b_g$ , $b_f$ , $b_o$ 는 각 게이트에서 사용되는 4개의 편향

### 입력 게이트

![alt text](inputgate.png)

$ g_t = tanh(W_{xg} x_t + W_{hg} h_{t-1} + b_g) $

$ i_t = \sigma (W_{xi} x_t + W_{hg} h_{t-1} + b_g) $

입력 게이트는 현재 정보를 기억하기 위한 게이트

$g_t$는 하이퍼볼릭탄젠트 함수를 지나 -1 ~ 1 사이의 값을 가지고, 

$i_t$는 시그모이드 함수를 지나 0 ~ 1 사이의 값을 지닌다

$i_t$의 값이 클수록 현재 시점의 입력을 많이 반영한다는 의미

입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정

### 삭제 게이트

![alt text](forgetgate.png)

$ f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-1} + b_f) $

삭제 게이트는 기억을 삭제하기 위한 게이트

$f_t$는 시그모이드 함수를 지나 0 ~ 1 사이의 값이 나오고, 해당 값이 곧 삭제 과정을 거친 정보의 양

0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것

삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미

### 셀 상태(장기 상태)

![alt text](cellstate2.png)

$ C_t = f_t \circ C_{t-1} + i_t \circ g_t $

셀 상태 $C_t$는 LSTM에서 장기 상태라고도 부름

입력 게이트에서 구한 $i_t$, $g_t$ 두 값을 원소별 곱(entrywise product)을 진행, 이 값이 이번에 선택된 기억할 값

입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더함, 이 값을 현재 시점 t의 셀 상태로, 다음 t+1 시점의 LSTM 셀로 넘겨짐

삭제 게이트의 출력값 $f_t$가 0이면 이전 시점의 셀 상태값 $C_{t-1}$ 또한 값이 0이 되면서

입력 게이트의 결과만이 현재 시점의 셀 상태값 $C_t$를 결정함, 이는 삭제 게이트가 완전히 닫히고 입력 게이트를 연 상태를 의미

반대로 입력 게이트의 $i_t$값이 0이면 현재 시점의 셀 상태값 $C_t$는 이전 시점의 셀 상태값 $C_{t-1}$의 값에 의존함

이는 입력 게이트를 완전히 닫고 삭제 게이트만을 연 상태를 의미

결과적으로 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정

### 출력 게이트와 은닉 상태(단기 상태)

![alt text](outputgateandhiddenstate.png)

$ o_t = \sigma (W_{xo} x_t + W_{ho} h_{t-1} + b_o) $

$ h_t = o_t \circ tanh(c_t) $

출력 게이트는 현재 시점 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값

해당 값은 현재 시점 t의 은닉 상태를 결정

은닉 상태를 단기 상태라고 하기도 함

은닉 상태는 장기 상태의 값이 하이퍼볼릭탄젠트 함수를 지나 -1 ~ 1 사이의 값

해당 값은 출력 게이트의 값과 연산되어, 값이 걸러지는 효과가 발생, 단기 상태의 값은 또한 출력층으로도 향함

## PyTorch 에서 사용

RNN 셀에서 LSTM으로 바꿔주면 사용가능하다

nn.RNN(input_dim, hidden_size, batch_fisrt=True)

->

nn.LSTM(input_dim, hidden_size, batch_fisrt=True)

In [22]:
input_dim = 10    # 입력의 크기
hidden_dim = 100  # 은닉 상태 크기
layer_dim = 1     # 은닉층 개수
output_dim = 1    # 출력의 크기

In [23]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()

        # 은닉 상태 크기
        self.hidden_dim = hidden_dim
        
        # 은닉층 개수
        self.layer_dim = layer_dim
        
        # LSTM 사용
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # 출력층
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # 은닉 상태 초기화
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # 셀 초기화
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 

        return out
    
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [24]:
# 예시 입력
seq_dim = 5  # 시퀀스 길이
batch_size = 3
input_data = torch.randn(batch_size, seq_dim, input_dim)
print(input_data.shape)
print(input_data)

torch.Size([3, 5, 10])
tensor([[[ 1.2039,  0.1609, -0.0956,  0.5675, -0.8150,  0.0067,  0.5392,
          -1.1186, -0.0790,  1.0845],
         [ 1.5401, -1.3782, -0.3112, -0.7934,  0.1500, -0.0784, -0.2337,
          -0.5998, -0.2178,  0.3963],
         [ 1.4409,  0.3609, -1.3276, -0.8240,  0.0078,  0.7238,  0.7650,
          -1.5663,  0.3778,  0.0276],
         [-1.8175, -1.2501,  0.2030,  0.5001, -1.0371,  0.0255,  0.0535,
          -1.3016, -1.9942, -0.9052],
         [ 2.2444, -0.3012,  0.3523,  0.3779,  0.8881,  0.7441, -1.0162,
          -0.1795,  0.4447, -1.2160]],

        [[ 0.2241,  0.1242, -0.3550,  0.4491, -0.3476, -0.4995, -0.2421,
           0.0030,  2.1632,  0.2112],
         [-1.3865, -0.5824, -0.6144,  0.7574, -0.5591, -1.0751,  0.3298,
          -0.0296, -1.8326, -0.3389],
         [ 0.0079,  1.5170,  1.4620, -0.4900, -0.9398, -0.7445,  1.0012,
          -0.1598, -0.1366,  0.6847],
         [-0.2086,  0.8868,  0.1302,  0.8639,  1.4595,  0.0491, -0.7165,
           0.0

In [25]:
# Forward pass
output = model(input_data)
print(output.shape)  # (batch_size, output_dim)

torch.Size([3, 1])
